In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [2]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical

In [3]:
(train_img,train_label),(test_img,test_label) = cifar10.load_data()

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10021598930056321465
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2909221684
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1229324065704101110
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

In [5]:
#print(train_img.shape) - > (50k ,32, 32,3)
train_img = train_img.astype('float32')/255.0
test_img = test_img.astype('float32')/255.0

train_label = to_categorical(train_label,10)
test_label = to_categorical(test_label,10) 

In [6]:
from keras.models import Model
from keras.layers import Input,Flatten,Dense,Conv2D,LeakyReLU,BatchNormalization,Dropout

In [7]:
input_layer = Input(shape=(32,32,3))

x = Conv2D(filters=32,
           kernel_size=3,
           strides=1, padding='same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=32,
           kernel_size=3,
           strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64,
           kernel_size=3,
           strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64,
           kernel_size=3,
           strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Flatten()(x)

x = Dense(units=128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate=0.5)(x)

output_layer = Dense(units=10,activation='softmax')(x)

model = Model(input_layer,output_layer)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 32)       128       
 hNormalization)                                             

In [9]:
from keras.optimizers import Adam
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy',optimizer = opt, metrics=['accuracy'])

d:\condaa\envs\tflow2.9\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [11]:
model.fit(train_img,train_label,
          batch_size = 32,
          epochs=10,
          shuffle=True,
          validation_data=(test_img,test_label))

Epoch 1/10
1563/1563 [==============================] - 24s 15ms/step - loss: 0.6269 - accuracy: 0.7805 - val_loss: 0.8849 - val_accuracy: 0.7075
Epoch 2/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.5985 - accuracy: 0.7902 - val_loss: 0.8112 - val_accuracy: 0.7207
Epoch 3/10
1563/1563 [==============================] - 29s 18ms/step - loss: 0.5718 - accuracy: 0.7975 - val_loss: 0.7929 - val_accuracy: 0.7317
Epoch 4/10
1563/1563 [==============================] - 30s 20ms/step - loss: 0.5509 - accuracy: 0.8040 - val_loss: 0.8842 - val_accuracy: 0.7110
Epoch 5/10
1563/1563 [==============================] - 31s 20ms/step - loss: 0.5206 - accuracy: 0.8154 - val_loss: 0.8548 - val_accuracy: 0.7316
Epoch 6/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.4988 - accuracy: 0.8234 - val_loss: 0.8206 - val_accuracy: 0.7405
Epoch 7/10
1563/1563 [==============================] - 38s 24ms/step - loss: 0.4872 - accuracy: 0.8274 - val_loss: 0.8714 -

In [12]:
loss,acc = model.evaluate(test_img,test_label,batch_size=1000)

10/10 [==============================] - 1s 68ms/step - loss: 0.9059 - accuracy: 0.7219
